In [1]:
import numpy as np
from scipy.optimize import minimize

### Задание 1

Найдите минимум следующих функций, используя методы минимизации, доступные в функции `scipy.optimize.minimize()`. Начальное приближение: $x_0 = \left[0, 0\right]$.

1. Функция Экли:
$$
	f(x,y) = -20\exp\left[-0.2\sqrt{0.5\left(x^2 + y^2\right)}\right] - \exp\left[0.5\left(\cos\left(2\pi x\right) + \cos\left(2\pi y\right)\right)\right] + e + 20
$$

2. Функция Била:
$$
		f(x,y) = \left(1.5 - x + xy\right)^2 + \left(2.25 - x + xy^2\right)^2 + \left(2.625 - x + xy^3\right)^2
$$

3. Функция Гольдшейна-Прайса:
$$
\begin{split}
		f(x,y) = \left[1 + \left(x + y + 1\right)^2\left(19 - 14x + 3x^2 - 14y + 6xy + 3y^2\right)\right] \cdot \\ \cdot \left[30 + \left(2x - 3y\right)^2 \left(18 - 32x + 12x^2 + 48y - 36xy + 27y^2\right)\right]
	\end{split}
$$

4. Функция Матьяса:
$$
		f(x,y) = 0.26\left(x^2 + y^2\right) - 0.48xy
$$

In [2]:
def f1(vars: np.ndarray) -> float:
    x, y = vars
    res = (
        -20 * np.exp(-.2 * (.5 * (x ** 2 + y ** 2)) ** .5)
        - np.exp(.5 * (np.cos(2 * np.pi * x)) + np.cos(2 * np.pi * y))
        + np.e + 20
    )
    return res

In [3]:
def f2(vars: np.ndarray) -> float:
    x, y = vars
    res = (
        (1.5 - x + x * y) ** 2 
        + (2.25 - x + x * y ** 2) ** 2 
        + (2.625 - x + x * y ** 3) ** 2)
    return res

In [4]:
def f3(vars: np.ndarray) -> float:
    x, y = vars
    res = (
        (1 + (x + y + 1) ** 2 
         * (19 - 14 * x + 3 * x ** 2 - 14 * y + 6 * x * y + 3 * y ** 2))
         * (30 + (2 * x - 3 * y) ** 2 
            * (18 - 32 * x + 12 * x ** 2 
               + 48 * y - 36 * x * y + 27 * y ** 2))
    )
    return res

In [5]:
def f4(vars: np.ndarray) -> float:
    x, y = vars
    res = .26 * (x ** 2 + y ** 2) - .48 * x * y
    return res


In [6]:
functions_list = [f1, f2, f3, f4]

In [16]:
for function in functions_list:
    res = minimize(
        fun=function,
        x0=[0, 0],
        method=''
    )
    print(res.fun, res.x, res.success)

-1.7633954784177526 [-3.01558360e-06 -2.86362133e-06] True
3.9119308661697416e-07 [2.99893098 0.49982984] True
30.000000012793894 [-0.60000137 -0.39999581] True
0.0 [0. 0.] True


### Задание 2

Пусть дана схема химических превращений:

$$
	A\xrightarrow{k_1}2B \xleftrightarrow[k_3]{k_2}C
$$

Необходимо определить с помощью генетического алгоритма и метода Нелдера-Мида (можно воспользоваться функцией `scipy.optimize.mnimize()`, указав соответствующее значение опционального аргумента `method`) константы скоростей реакций: $k_1$, $k_2$ и $k_3$, если известно, что к моменту времени $t=1 (\text{c})$ концентрации компонентов равны: $C_A = 0.1423$; $C_B = 1.5243$; $C_C = 0.5956\  \mathrm{моль/л}$.

Начальные условия: $C_A(0)=1.0$; $C_B=0.0$; $C_C=0.5\ \mathrm{моль/л}$.
Область поиска для всех констант ограничьте интервалом $[0;\ 2]$.

In [19]:
from scipy.integrate import solve_ivp
from genetic_algorithm import genetic_algorithm as ga

In [17]:
def kinetic(
        t: float,
        conc: np.ndarray,
        k: np.ndarray
) -> tuple[float, float, float]:
    a, b, c = conc
    k1, k2, k3 = k
    dca = -k1 * a
    dcb = 2 * (k1 * a - k2 * b ** 2 + k3 * c)
    dcc = k2 * b ** 2 - k3 * c
    return dca, dcb, dcc

In [20]:
def objective_function(
        k: np.ndarray,
        c_exist: np.ndarray,
        c0: np.ndarray,
        t: tuple[float, float]
) -> float:
    sol = solve_ivp(
        fun=kinetic,
        t_span=t,
        y0=c0,
        args=(k, )
    )
    c_calc = sol.y[:, -1]
    return ((c_exist - c_calc) ** 2).sum()

In [24]:
c_exist = np.array(
    [
        .1423, 1.5243, .5956
    ]
)
c_init = np.array(
    [
        1., .0, .5
    ]
)
t = .0, 1.

In [30]:
bounds = [[0, 2] for _ in range(3)]
res_ga = ga(
    bounds=bounds,
    func=objective_function,
    args=(
        c_exist,
        c_init,
        t    
    )
)

In [32]:
res_ga[0]

[1.951862254377227, 0.6028170777233074, 1.9227586245187753]

In [28]:
res = minimize(
    fun=objective_function,
    x0=[1., 1., 1.],
    method='Nelder-Mead',
    args=(
        c_exist,
        c_init,
        t
    )
)

In [29]:
res

       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: 1.1149325141431506e-09
             x: [ 1.950e+00  3.318e-01  8.320e-01]
           nit: 66
          nfev: 116
 final_simplex: (array([[ 1.950e+00,  3.318e-01,  8.320e-01],
                       [ 1.950e+00,  3.318e-01,  8.320e-01],
                       [ 1.950e+00,  3.318e-01,  8.320e-01],
                       [ 1.950e+00,  3.319e-01,  8.321e-01]]), array([ 1.115e-09,  1.132e-09,  1.133e-09,  1.134e-09]))